### Fase 1 - Configuración global y constantes
- Librerias necesarias
- Definicion de constantes globales: comportamiento y apariencia

In [144]:
# Librerias
import copy 
import random 
import time

In [145]:
# Apariencia
AVATAR_P1 = "🐱"
AVATAR_P2 = "🐭"
CASILLA = "⬜"
PARED = "⬛"
CASILLA_SALIDA = "⏬"
DIVISOR = ""

# Casillas caminables
CAMINABLES = [CASILLA, AVATAR_P2, CASILLA_SALIDA]

# Constantes del juego 
MAX_TURNOS = 15
FILAS = 8
COLUMNAS = 7 
SEARCH_DEPTH = 4 # Profundidad de busqueda
TIMER_SMART = 1 # Turnos para "activar" la inteligencia del raton

# Direcciones 
DIRECCIONES = [
    (-1, 0), (1, 0), (0, -1), (0, 1), # Direcciones en cruz 
    (1, 1), (-1, 1), (1, -1), (-1, -1), # Direcciones diagonales
]

# Coordenadas de la salida
SALIDAS_COORD = [(0, COLUMNAS -1), (0, 0)] 

### Fase 2 - Creacion y visualización del tablero
    Funciones: 
- **Crear partida aleatoria( ):** Genera un nuevo tablero y coloca al gato y al ratón en sus posiciones iniciales. 

- **Imprimir tablero ( ):** Muestra el estado actual del juego en la consola. 

In [146]:
def crear_partida_aleatoria():
    """
    Crea un nuevo tablero con el gato y el ratón en posiciones aleatorias y válidas.
    """
    tablero = [[CASILLA for _ in range(COLUMNAS)] for _ in range(FILAS)]

    for columna in range(COLUMNAS):
        tablero[0][columna] = PARED

    for salida in SALIDAS_COORD:
        tablero[salida[0]][salida[1]] = CASILLA_SALIDA
    
    gato_fila = random.randint(1, 1) # Gato se coloca en la fila 1 (la fila 0 es la salida)
    gato_col = random.randint(0, COLUMNAS - 1) 
    
    raton_fila = random.randint(FILAS - 2, FILAS - 1) # Ratón aparece en las últimas dos filas (FILAS-1 o FILAS-2)
    raton_col = random.randint(0, COLUMNAS - 1)
    
    while (gato_fila, gato_col) == (raton_fila, raton_col):
        # Si coinciden, solo regeneramos la del ratón para simplificar
        raton_fila = random.randint(FILAS - 2, FILAS - 1)
        raton_col = random.randint(0, COLUMNAS - 1)
        
    tablero[gato_fila][gato_col] = AVATAR_P1
    tablero[raton_fila][raton_col] = AVATAR_P2
    
    print("¡Comienza el juego Gato vs. Ratón! 🐱🐭 Se ha generado una nueva partida aleatoria")
    return tablero

In [147]:
# Funcion para visualizar el tablero:
def imprimir_tablero(tablero_actual):
    """
    Muestra el estado actual del juego en la consola. 

    Argumentos: 
        tablero_actual: Una lista de listas que representa al tablero. Cada sublista es una fila del tablero.
    """
    for f in range(FILAS): # Itera por el índice de la fila
        print_fila = []
        for c in range(COLUMNAS): # Itera por el índice de la columna
            actual_coord = (f, c)
            casilla_tablero = tablero_actual[f][c]

            # Si la coordenada corresponde a la salida y esta vacía
            if actual_coord in SALIDAS_COORD and casilla_tablero == CASILLA:
                print_fila.append(CASILLA_SALIDA) # Se dibuja la salida
            else:
                # Si no, se imprime gato, raton, o casilla vacia 
                print_fila.append(casilla_tablero)

        print(DIVISOR.join(print_fila)) # "Divisor" es una variable global definida mas arriba

### Fase 3 - Mecánicas del Juego
    Definición de reglas y acciones básicas del juego. 
- **find_pos_jugadores( ):** ubica a ambos jugadores en el tablero.
- **movimientos_ok( ):** Determina los movimientos válidos.
- **moverse( ):** Actualiza el tablero aplicando el movimiento del jugador.
- **verificar_estado_juego( ):** Comprueba el estado del juego (victoria o empate)

In [148]:
# Funcion para determinar posicion actual de los jugadores en el tablero:

def find_pos_jugadores(tablero_actual):
    """
    Encuentra la posición actual del gato y el ratón en el tablero.

    Args:
        tablero_actual (lista): El estado actual del tablero (lista de listas).

    Returns:
        tuple: Una tupla conteniendo (gato_pos, raton_pos).
        Cada posición es una tupla (fila, columna) o None si no se encuentra. """

    gato_pos = None
    raton_pos = None

    # Encontramos la posicion actual (P1) (P2). Se recorre el tablero buscando sus "avatar"
    for f in range(FILAS):        # Itera sobre las filas
        for c in range(COLUMNAS): # Itera sobre las columnas
            if tablero_actual[f][c] == AVATAR_P1:
                gato_pos = (f, c)
            elif tablero_actual[f][c] == AVATAR_P2: 
                raton_pos = (f, c)

    # Devuelve las posiciones encontradas
    return gato_pos, raton_pos 

In [149]:
## Funcion movimientos validos - Coordenadas a las que el jugador PODRIA moverse (aun no es el movimiento real)
def movimientos_ok(posicion_actual, tablero_actual):
    """
    Genera una lista de todos los movimientos válidos, desde una posición dentro del tablero actual. 

    Args:
        posicion_actual (tupla): Coordenadas (fila, columna) actuales del jugador.
        tablero_actual (lista): El estado actual del tablero (lista de listas).

    Returns:
        lista: Una lista de tuplas, donde cada tupla es una coordenada (fila, columna) a la que el jugador puede moverse legalmente.
    """
    movimientos = [] # Aqui se guardan todas las posiciones validas a las que se puede mover
    fila_actual, col_actual = posicion_actual # Desempaquetar la posicion actual

    for df, dc in DIRECCIONES: 
        nueva_fila = fila_actual + df # Se calcula la nueva fila
        nueva_col = col_actual + dc # Se calcula la nueva columna
        nueva_pos = (nueva_fila, nueva_col) # Guardamos la nueva posición para que sea más fácil de leer

        # Para si la nueva posicion esta dentro de los limites del tablero:
        if not ((0 <= nueva_fila < FILAS) and (0 <= nueva_col < COLUMNAS)): continue

        # Si las nuevas posiciones no estan en los posibles movimientos caminables, se ignora este paso
        if tablero_actual[nueva_fila][nueva_col] not in CAMINABLES: continue

        # Si la nueva posición esta en la primera fila, y no pertenece a las coordenadas de la salida, se ignora este paso
        if nueva_fila == 0 and nueva_pos not in SALIDAS_COORD: continue
            
        movimientos.append(nueva_pos)

    return movimientos

In [150]:
# Pedir la entrada con input, validar si la entrada es correcta, y actualizar el tablero. 

# Funcion para realizar movimiento en el tablero:
def moverse(tablero_actual, jugador, old_pos, new_pos):
    """
    Crea un nuevo estado del tablero después de que un jugador realiza un movimiento.

    Args:
        tablero_actual (list): El tablero antes del movimiento.
        jugador (str): El avatar del jugador que se mueve.
        old_pos (tuple): La posición (fila, columna) antigua del jugador.
        new_pos (tuple): La posición (fila, columna) nueva del jugador.

    Returns:
        list: Un NUEVO tablero (lista de listas) con el movimiento realizado.
    """
    # Se crea una copia del tablero para no modificar el original directamente.
    # Se utiliza la funcion copy.deepcopy para crear una copia nueva e independiente de la anterior
    # Con esto aseguramos que los estados hipoteticos no afecten al tablero original. 
    tablero_nuevo = copy.deepcopy(tablero_actual)

    # Se borra al jugador de su posicion anterior
    tablero_nuevo[old_pos[0]][old_pos[1]] = CASILLA

    # Colocar al jugador en su nueva posicion
    tablero_nuevo[new_pos[0]][new_pos[1]] = jugador

    return tablero_nuevo

In [151]:
# Funcion verificar estado del juego
def verificar_estado_juego(raton_pos, turno_numero):
    """
    Comprueba si el juego ha terminado y determina el resultado.

    Args:
        raton_pos (tupla): Posición actual (fila, columna) del raton o None si ha sido atrapado. 
        turno_numero (int): El número del turno actual en el juego.

    Returns:
        str: Una cadena de texto que indica el estado del juego.
    """

    # Condicion 1: Victoria del gato (P1) - Si la posicion del ratón is None.  
    if raton_pos is None:
        return "Ganador: 🐈" #El gato "atrapo" al ratón. 
    
    # Condicion 2: Victoria del Ratón (P2) - Si el ratón ha llegado a la "salida" del tablero
    elif raton_pos in SALIDAS_COORD:
        return "Ganador: 🐁" # El ratón "escapó"
    
    # Condicion 3: Empate por límite de turnos - Si se cumple el numero máximo de turnos y nadie ha ganado
    elif turno_numero >= MAX_TURNOS:
        return "Empate" # Límite de turnos alcanzado, ratón sobrevivió pero no escapó
    
    # Si ninguna de las condiciones anteriores se cumple, el juego continúa
    return "Partida en curso"

### Fase 4 - Cerebro del juego - Algoritmo MiniMax
    Las funciones implementan el algoritmo MiniMax para la toma de decisiones:
- **evaluar_partida( ):** Asigna una puntuación numérica a un estado del tablero. Es el "criterio" de evaluacion.
- **minimax( ):** Explora recursivamente los posibles movimientos futuros. 
- **find_best_move( ):** Utiliza la funcion minimax para analizar todos los movimientos inmediatos y seleccionar el que ofrece la mejor puntuación. 

In [152]:
def evaluar_partida(tablero_actual, depth):
    """
    Asigna una puntuación numérica a un estado del tablero desde la perspectiva del gato (maximizador).
    Un puntaje alto favorece al gato, un puntaje bajo favorece al ratón.

    Args:
        tablero_actual (list): El estado del tablero a evaluar.
        depth (int): La profundidad actual en la búsqueda Minimax. Se usa para premiar/penalizar la rapidez con la que se alcanza un estado terminal.

    Returns:
        int:La puntuación del estado del tablero.
    """
    gato_pos, raton_pos = find_pos_jugadores(tablero_actual) # Obtener posiciones de ambos jugadores

    evaluacion = 0 # Se inicializa la evaluación en 0 
    retardo = SEARCH_DEPTH - depth # Se miden cuantos turnos "tardo" la simulación en llegar, recompensa la rapidez

    # Caso 1: Gana el gato - 
    if raton_pos is None: # Si el ratón "no está", el gato gana
        evaluacion = 10000 - retardo
        return evaluacion
    
    # Caso 2: Victoria del ratón - 
    if raton_pos in SALIDAS_COORD:
        evaluacion = -10000 + retardo
        return evaluacion

    # Si no termina el juego: 
    # Puntuación por dirigirse haca el ratón, 
    distancia_raton_gato = abs(gato_pos[0] - raton_pos[0]) + abs(gato_pos[1] - raton_pos[1])
    evaluacion -= distancia_raton_gato
    
    # Distancia MÍNIMA del ratón a CUALQUIERA de las salidas
    distancia_min_raton = min(abs(raton_pos[0] - salida[0]) + abs(raton_pos[1] - salida[1]) for salida in SALIDAS_COORD) * 20 
    evaluacion += distancia_min_raton
    
    # Devuelve la puntuacion
    return evaluacion

In [153]:
# Función MiniMax
# Se detiene si alcanza el limite de profundidad (depth)
# Se detiene si el juego ya ha terminado en ese futuro 

# Funcion para evaluar los puntajes 

def minimax(tablero_actual, turno_numero, depth, turno_gato):
    """
    Implementa el algoritmo Minimax de forma recursiva para encontrar la mejor
    puntuación posible desde el estado actual del tablero, hasta una profundidad dada.

    Args:
        tablero_actual (list): El estado actual del tablero (lista de listas).
        turno_numero (int): El número del turno actual en el juego real.
        depth (int): La profundidad de búsqueda restante. 
        turno_gato (bool): True si es el turno del Gato (maximizador) en la
                        simulación actual, False si es el turno del Ratón (minimizador).

    Returns:
        int: La puntuación evaluada para el estado actual del tablero.
    """

    gato_pos, raton_pos = find_pos_jugadores(tablero_actual)
    estado_juego = verificar_estado_juego(raton_pos, turno_numero)

    if depth == 0 or estado_juego != "Partida en curso": # Limite de profundidad o termino el juego
        return evaluar_partida(tablero_actual, depth) # Imprime el tablero final y devuelve la puntuación

    if turno_gato: # Turno del gato (maximizador)
        evaluacion = float("-inf") # Inicia en -infinito para sumar a partir de ahi. 
        avatar = AVATAR_P1
        optimizacion = max
        posicion = gato_pos

    # Turno del Ratón (Minimizador)
    else:
        evaluacion = float("inf") # Inicia en infinito para restar a partir de ahi. 
        avatar = AVATAR_P2
        optimizacion = min
        posicion = raton_pos

    # Si el raton ya no esta en el tablero, termina la evaluación 
    if not posicion:
        return evaluar_partida(tablero_actual, depth)
        
    mov_posibles = movimientos_ok(posicion, tablero_actual) 

    if not mov_posibles: 
        return evaluar_partida(tablero_actual, depth)
        
    for mov in mov_posibles: # Crea un tablero hipotetico donde ya se realizo el movimento, y se llama a si misma para analizar el nuevo tablero.
        tablero_hipotetico = moverse(tablero_actual, avatar, posicion, mov)
        evaluacion = optimizacion(evaluacion, minimax(tablero_hipotetico, turno_numero, depth - 1, not turno_gato))
            
    return evaluacion

In [154]:
def find_best_move(tablero_actual, turno_numero, turno_gato):
    """
    Analiza los movimientos posibles para el jugador actual utilizando el algoritmo
    MiniMax para seleccionar el mejor movimiento posible.

    Args:
        tablero_actual (list): El estado actual del tablero en el juego real.
        turno_numero (int): El número del turno actual.
        turno_gato (bool): True si es el turno del gato (maximizador), False si es
                        el turno del ratón (minimizador).

    Returns:
        tuple or None: La coordenada (fila, columna) del mejor movimiento encontrado,
                    o None si no hay movimientos posibles.
    """
    
    gato_pos, raton_pos = find_pos_jugadores(tablero_actual)
    mejor_movimiento = None

    if turno_gato:
        posibles_movimientos = movimientos_ok(gato_pos, tablero_actual)
        mejor_puntuacion_conocida = float("-inf") # Gato maximiza
        avatar_jugador = AVATAR_P1
        # En la simulación de minimax, el siguiente en jugar será el ratón
        simular_turno_gato_para_minimax = False 
        posicion_jugador_actual = gato_pos

    else: # Turno del ratón
        posibles_movimientos = movimientos_ok(raton_pos, tablero_actual)
        mejor_puntuacion_conocida = float("inf") # Ratón minimiza (la puntuación del gato)
        avatar_jugador = AVATAR_P2
        # En la simulación de minimax, el siguiente en jugar será el gato
        simular_turno_gato_para_minimax = True
        posicion_jugador_actual = raton_pos

    if not posibles_movimientos: # Si no hay movimientos posibles
        return None
    
    # Movimiento de respaldo por si todos tienen la misma puntuación
    mejor_movimiento = random.choice(posibles_movimientos) 

    for mov in posibles_movimientos:
        tablero_hipotetico = moverse(tablero_actual, avatar_jugador, posicion_jugador_actual, mov)

        puntuacion_evaluada = minimax(tablero_hipotetico, turno_numero, SEARCH_DEPTH, simular_turno_gato_para_minimax)
        # print(F" --> moverse a {mov}: Puntiación calculada por Minimax = {puntuacion_evaluada}")

        if turno_gato: # Gato (Maximizador)
            if puntuacion_evaluada > mejor_puntuacion_conocida:
                mejor_puntuacion_conocida = puntuacion_evaluada
                mejor_movimiento = mov
        else: # Ratón (Minimizador)
            if puntuacion_evaluada < mejor_puntuacion_conocida:
                mejor_puntuacion_conocida = puntuacion_evaluada
                mejor_movimiento = mov

    if turno_gato:
        print(f"-- {AVATAR_P1} eligio moverse a {mejor_movimiento} (puntuacion final: {mejor_puntuacion_conocida}) ---- \n")
    else:
        print(f"-- {AVATAR_P2} eligio moverse a {mejor_movimiento} (puntuacion final: {mejor_puntuacion_conocida}) ---- \n")

    return mejor_movimiento

### Fase 5 - Simulación y Ejecución del Juego
    La función: jugar_partida( ):
- Inicia el tablero
- Controla los turnos de los jugadores
- Llama a **find_best_move** para obtener movimientos inteligentes. 
- Actualiza y muestra el tablero despues de cada jugada.
- Finaliza el juego cuando se cumple una condición. (victoria o empate)

Finalmente, se ejecuta jugar_partida( ) para iniciar la simulacion. 

In [155]:
def jugar_partida():
    """
    Inicia y gestiona una partida completa del juego Gato vs. Ratón.

    Esta función se encarga de:
    - Crear un tablero de juego aleatorio.
    - Controlar el flujo de turnos entre el gato y el ratón.
    - Obtener los movimientos de los jugadores.
    - Actualizar el estado del tablero después de cada movimiento.
    - Imprimir el tablero y el estado del juego en cada paso.
    - Detectar y anunciar el final del juego.
    """

    tablero_juego = crear_partida_aleatoria()
    turno_numero = 1
    jugador_actual = AVATAR_P1 #El gato inicia el juego
    estado_juego = "Partida en curso"

    imprimir_tablero(tablero_juego)

    while True:
        # --- 1. VERIFICAR ESTADO GENERAL DEL JUEGO ---
        gato_pos_actual, raton_pos_actual = find_pos_jugadores(tablero_juego)
        estado_juego = verificar_estado_juego(raton_pos_actual, turno_numero)
        
        if estado_juego != "Partida en curso":
            print(f"\nFin del juego - Resultado: {estado_juego}")
            imprimir_tablero(tablero_juego)
            return #Termina la funcion jugar partida
        print(f"\nRonda {turno_numero}  Jugador: {jugador_actual}")

        # --- 2. LÓGICA DE TURNOS SEPARADA ---\n",
        if jugador_actual == AVATAR_P1:  # TURNO DEL GATO
            next_pos = find_best_move(tablero_juego, turno_numero, True) #True para indicar turno del gato

            if next_pos is None:
                print("El {AVATAR_P1} no puede moverse. Turno perdido.")
                continue 
            else:
                tablero_juego = moverse(tablero_juego, AVATAR_P1, gato_pos_actual, next_pos)
                imprimir_tablero(tablero_juego)
                time.sleep(0.1)

                # Se verifica si el gato gano despues de moverse
                _, raton_despues_del_ataque = find_pos_jugadores(tablero_juego) # Solo se necesita la posicion del raton
                if raton_despues_del_ataque is None:
                    print(f"\n¡EL GATO HA ATRAPADO AL {AVATAR_P2}!")
                    estado_juego = "Ganador: 🐈"
                    break

            jugador_actual = AVATAR_P2
            # print(f"\n{'=' * 20}\n")
            continue

        else: # Turno del ratón 
            next_pos = raton_pos_actual

            if turno_numero <= TIMER_SMART:
                print("El ratón se mueve al azar...")
                mov_seguros_raton = movimientos_ok(raton_pos_actual, tablero_juego)
                if mov_seguros_raton:
                    next_pos = random.choice(mov_seguros_raton)
                    
            else:
                # print("El ratón se mueve de forma inteligente...")
                next_pos = find_best_move(tablero_juego, turno_numero, False)

            if next_pos is None:
                print("El Ratón no puede moverse. Pierde su turno.")
                imprimir_tablero(tablero_juego)
            elif next_pos == raton_pos_actual: 
                print("El Ratón no puede moverse. Pierde su turno.")
                imprimir_tablero(tablero_juego)
            else:
                tablero_juego = moverse(tablero_juego, AVATAR_P2, raton_pos_actual, next_pos)
                imprimir_tablero(tablero_juego)

                if next_pos in SALIDAS_COORD: # El ratón llego a una salida
                    print(f"\n¡EL {AVATAR_P2} SE HA ESCAPADO!")
                    estado_juego = "Ganador: 🐁"
                    break
            
            jugador_actual = AVATAR_P1
            turno_numero += 1
            # time.sleep(1)
            # print(f"\n{'=' * 20}\n")

    # Cuando el juego termina, "estado_juego" tiene el resultado. 
    print (f"Fin del juego - Resultado: {estado_juego}")

    imprimir_tablero(tablero_juego) # Se imprime el tablero final definitivo

In [156]:
jugar_partida()

¡Comienza el juego Gato vs. Ratón! 🐱🐭 Se ha generado una nueva partida aleatoria
⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜🐱
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐭

Ronda 1  Jugador: 🐱
-- 🐱 eligio moverse a (2, 6) (puntuacion final: 99) ---- 

⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐱
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐭

Ronda 1  Jugador: 🐭
El ratón se mueve al azar...
⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐱
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜🐭⬜
⬜⬜⬜⬜⬜⬜⬜

Ronda 2  Jugador: 🐱
-- 🐱 eligio moverse a (3, 6) (puntuacion final: 119) ---- 

⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐱
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜🐭⬜
⬜⬜⬜⬜⬜⬜⬜

Ronda 2  Jugador: 🐭
-- 🐭 eligio moverse a (5, 4) (puntuacion final: 98) ---- 

⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜🐱
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜🐭⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜

Ronda 3  Jugador: 🐱
-- 🐱 eligio moverse a (3, 5) (puntuacion final: 99) ---- 

⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜🐱⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜🐭⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜

Ronda 3  Jugador: 🐭
-- 🐭 eligio moverse a (4, 3) (puntuacion final: 58) ---- 

⏬⬛⬛⬛⬛⬛⏬
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜🐱⬜
⬜⬜⬜🐭⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜

Ronda 4  Jugador: 🐱